#### TODO
* parse my markdownish headlining
  - 1st level heading: single line of text, no leading starts, maybe short too, no leading spaces
  - 1st level heading if already has a leading star
  - 2nd level heading: leading `o ` , say one line long
  - 2nd level heading '--' and had leading '*'
  - 2nd level heading if leading '**'
  - 3rd level heading: leading ` -`
  - 3rd level heading: leading '***'
  - 4th level heading: leading ' --' and no leading star
  - 4th level heading: if leading '****'
  - 5th level          leading ' ----`
* TODO: but org-roam-ui removes *'s, seems to render markdown bullets?, so '-' becomes a bullet?
* org-roam-ui ignores #+OPTIONS: H:0 num:0
* TODO: some way to control org-roam UI's org rendering?

BUT check a few notes to see how consistent I was
  
* Convert British spelling to American?
I only found one example where this split a tag: "optimization".
quick hack:
https://stackoverflow.com/questions/42329766/python-nlp-british-english-vs-american-english
https://stackoverflow.com/questions/18840640/python-2-7-find-and-replace-from-text-file-using-dictionary-to-new-text-file

* make org-id from note's "date created" field if available.  If not, use paper year + something.  Also, just use fractions of UTC clock from current computer.  

In [1]:
from pathlib import Path
import re
from bibtexparser.bparser import BibTexParser
from dateutil.parser import parse
import string
from collections import defaultdict
from nltk.corpus import wordnet

In [2]:
refDir = Path("~/ref").expanduser()
bibDirBase = refDir

bibInNm = "deepSolarDOE"
bibInNm = "newTechAdopt"
bibInNm = "energy"

bibInFNm = bibDirBase / f"{bibInNm}.bib"
noteOutDir = Path("~/share/tmp_org_roam_test").expanduser()
noteOutDir.mkdir(parents=True, exist_ok=True)

# for colapsing/replacing whitespace
matchWhiteSpace_regexp = re.compile(r"\s+")
# for de-orgifying orig .bib comments
matchLeadStars_regexp = re.compile(r"\s+")

print(bibInFNm)

C:\Users\scott\ref\energy.bib


In [3]:
# Read source .bib as string so can handle parsing in a separate step, avoiding
# error throwing on unquoted .bib fields to right of '='.
# (If bibtexparser were to write them out, they would be restored as in the
# original bib file).

with open(bibInFNm, encoding="utf8") as bibtex_file:
    bibtex_str = bibtex_file.read()

bp = BibTexParser(interpolate_strings=False)
bib_database = bp.parse(bibtex_str)

### Write bib fields to org-roam notes

In [4]:
try:
    _ = wordnet.morphy("test")
except:
    print("Installing nltk data")
    import nltk
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    print("Done installing nltk data")

In [5]:
# Collect all bibtex keys in the .bib file, and make roam_tags strings
bibkeysAll = set()
keywordsCntAll = defaultdict(lambda: 0)
keywordsItem = defaultdict(lambda: [])
remPunctTable = str.maketrans('', '', string.punctuation)

# TODO: is the try below neeeded?  Isnt' 
# try:
#     _ = wordnet.morphy("test")
# except:
#     print("Installing nltk data")
#     import nltk
#     nltk.download('wordnet')
#     print("Done installying nltk data")

# Collect all bibkeys and (normalized) keywords for each item
for bibitem in bib_database.entries:
    bibkey = bibitem['ID']
    bibkeysAll.add(bibkey)

    if "keywords" in bibitem:
        # convert various .bib keyword list formats to OR format
        for kw_phrase in re.split(', |; |,|;', bibitem['keywords']):
            if len(kw_phrase) > 0:
                # normalize to lowercase, no punctuation, singular, no spaces
                kw_phrase = kw_phrase.translate(remPunctTable).lower().split()
                for idx, word in enumerate(kw_phrase):
                    if (word_singular := wordnet.morphy(word)) is not None:
                        kw_phrase[idx] = word_singular

                kw_phrase = "_".join(kw_phrase)
                keywordsItem[bibkey] += [kw_phrase]
                keywordsCntAll[kw_phrase] += 1

# Remove rare keywords and then make a roam_tags string for each bibkey
nOccurMin = 3
keepKeywords = set({kw: count for kw, count in keywordsCntAll.items()
                    if count >= nOccurMin}.keys())

roam_tags_str = dict()
for bibkey in bibkeysAll:
    kws = keywordsItem[bibkey]
    kwsItemKeep = keepKeywords.intersection(kws)
    roam_tags_str[bibkey] = f'#+roam_tags: {" ".join(kwsItemKeep)}'

print(f"keeping {len(keepKeywords)} of {len(keywordsCntAll)} unique keywords")

keeping 287 of 3322 unique keywords


In [6]:
# Translate selected .bib entries to org-roam note syntax, and write notes

def fix_comment_stars(commentStr):
    """Replace leading stars with 'o's.  Bib used '*'s which may
    result in inadvertently collapsed .org headlines"""

    outlns = []
    for ln in commentStr.split('\n'):
        noLstars = ln.lstrip('*')
        outlns += ["o" * (len(ln) - len(noLstars)) + noLstars]

    return "\n".join(outlns)


def bibitem_to_OR_note(bibitem):
    """Makes org-roam note text from a .bib file item.  The note contains 
    the .bib comment, abstract and metadata.  The filename, OR title and key
    are the .bib citekey."""

    bibkey = bibitem['ID']

    noteLns = []
    if "title" in bibitem:
        noteLns += [f"={bibitem['title']}="]

    authChunks = []
    if "author" in bibitem:
        authChunks += [bibitem['author']]

    if "year" in bibitem:
        authChunks += [f"({bibitem['year']})"]

    if len(authChunks) > 0:
        authStr = " ".join(authChunks)
        noteLns += [f"/{authStr}/"]

    if "comment" in bibitem:
        commentTxt = bibitem['comment']
        words = re.findall(r'\w+', commentTxt)
        words = set([w.translate(remPunctTable) for w in words])

        for citedKey in words.intersection(bibkeysAll):
            linkStr = f"[[file:{citedKey}.org][{citedKey}]]"
            commentTxt = commentTxt.replace(citedKey, linkStr)

        noteLns += ["", fix_comment_stars(commentTxt)]

    if "abstract" in bibitem:
        noteLns += ["", "* Abstract", bibitem['abstract']]

    noteLns += ["", "* Org-Roam Metadata", f"#+title:{bibkey}"]

    if "timestamp" in bibitem:
        try:
            dt = parse(bibitem['timestamp'])
            timeStr = dt.strftime('%Y-%m-%d %a')
        except:
            timeStr = "?"

        noteLns.append(f"#+created: [{timeStr}]")

    # TODO: instead of a roam key man an org-id for this note, and make any links to it point use the org-id
    #       See cell below for how to make the org-id
    # TODO: figure out why ORv2 has a ROAM_REFS entry that has a cite:<bibtexkey> thing in it.
    noteLns += [f"#+roam_key: cite:{bibkey}",
                roam_tags_str[bibkey]]

    return noteLns


# Write a separate org-roam note file for each .bib entry
for bibitem in bib_database.entries:
    noteLns = bibitem_to_OR_note(bibitem)

    noteOutFNm = noteOutDir / f"{bibitem['ID']}.org"
    with open(noteOutFNm, 'w', encoding="utf8") as fh:
        # print(f"Writing notes to {noteOutFNm}")
        for ln in noteLns:
            fh.write('%s\n' % ln)

print("\nDone.")


Done.


In [7]:
# the org-id format with (org-id-method 'ts)
# TODO: get ID date from bibtex data created field or year of publication + something else.  Use computers's fractional time.
from datetime import datetime
my_date = datetime.now()
# %Y%m%dT%H%M%S.%6N
# TODO: do OR's ID's have the 'T' in the middle?
print(my_date.strftime('%Y%m%dT%H%M%S.%f%z'))

20220524T092246.069718


### Prototype code below.  Don't run.

#### pybtex parser string examples

In [8]:
# from pybtex.database.input import bibtex
# bibitem['keywords']="CRPS, Diagnostic tools  , Evaluation framework, Ignorance Score, Probabilistic solar forecasting, Scoring rules"
# bibkws = bibitem['keywords'].split(", ")
# bibkws

# s = 'Probabilistic    solar forecasting'
# kws_underscore = []
# for kw in bibitem['keywords'].split(", "):
#     kws_underscore += [matchWhiteSpace_regexp.sub("_", kw.strip())]

# keywords = " ".join(kws_underscore)
# keywords

In [9]:
# bibtex = """@STRING{ jean = "Jean"}

# @ARTICLE{Cesar2013,
#   author = jean # { César},
#   title = {An amazing title},
#   year = {2013},
#   month = jan,
#   volume = {12},
#   pages = {12--23},
#   journal = {Nice Journal},
# }
# """

# bp = BibTexParser(interpolate_strings=False)
# bib_database = bp.parse(bibtex)
# bib_database.entries[0]
# as_text(bib_database.entries[0]['author'])

#### BibTeX::Parser
I wasn't able to get this to not collapse whitespace, so below is the hack to get around that.  But it turned out that bibtexparser could do the job w/o the hack.

In [10]:
# bibIn = bibtex.Parser().parse_file(bibInFNm)

# # get keys and whether or not it has a comment
# bibkeyFields = dict()
# for bibkey, bibitem in bibIn.entries.items():
#     if 'comment' in bibitem.fields:
#         bibkeyFields[bibkey] = True

In [11]:
# keysWithComments = list(bibkeyFields.keys())
# nextKey = keysWithComments.pop(0)
# p = re.compile(f"^@.*{nextKey},")
# lnum=1
# with open(bibInFNm, encoding="utf8") as fileHandler:
#     try:
#         for ln in fileHandler:
#             ln = ln.rstrip()
#             if p.match((ln)):
#                 print(lnum, ln)
#                 nextKey = keysWithComments.pop(0)
#                 p = re.compile(f"^@.*{nextKey},")
#             lnum += 1
#     except:
#         print(f"trouble with line: {ln}")